In [2]:
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
import tensorflow_addons as tfa
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))


import automatic_speech_recognition as asr
import time
from datetime import datetime
import argparse
import pickle

In [3]:
import numpy as np
import pandas as pd


In [4]:
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [5]:
from automatic_speech_recognition.model.quartznet import QUARTZNET_LAYERS

In [6]:
import horovod.tensorflow.keras as hvd

In [7]:
#%load_ext tensorboard
#%tensorboard --logdir=./models/ --port=32779

In [8]:
%pdb off

Automatic pdb calling has been turned OFF


## Train

In [9]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [10]:
# Initialize Horovod
hvd.init()
# Pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

In [11]:
def get_pipeline(model, optimizer=None):
    alphabet = asr.text.Alphabet(lang='en')
    features_extractor = asr.features.FilterBanks(
        features_num=64,
        standardize=None,
        winlen=0.02,
        winstep=0.01,
    )
    if not optimizer:
        optimizer = tf.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
    decoder = asr.decoder.GreedyDecoder()
    pipeline = asr.pipeline.CTCPipeline(
        alphabet, features_extractor, model, optimizer, decoder
    )
    return pipeline

In [14]:
def train_model(filename, dataset_idx, val_dataset_idx=None, initial_lr=0.001, batch_size=10, epochs=25, tensorboard=False, restart_filename=None,
               is_mixed_precision=False, n_blocks=1, decay=1.0):
    basename = os.path.basename(filename).split('.')[0]
    model_dir = os.path.join(os.path.dirname(filename), basename + '_train')
    os.makedirs(model_dir, exist_ok=True)
    
    model = asr.model.get_quartznet(64, 29, is_mixed_precision=is_mixed_precision, num_b_block_repeats=n_blocks)

    if restart_filename:
        model.load_weights(restart_filename)
    
    initial_lr_global = initial_lr * hvd.size()
    
    dataset = asr.dataset.Audio.from_csv(dataset_idx, batch_size=batch_size,
                                         use_filesizes=True, group_size=hvd.size(), rank=hvd.rank())
    dataset.sort_by_length()
    
    dataset._references = dataset._references[:batch_size*1]
    dataset._indices = np.arange(len(dataset))
    
    dataset.shuffle_indices()

    print(f'Group size: {hvd.size()} rank {hvd.rank()} got {len(dataset)} batches')

    if val_dataset_idx:
        val_dataset = asr.dataset.Audio.from_csv(val_dataset_idx, batch_size=batch_size, use_filesizes=True,
                                                group_size=hvd.size(), rank=hvd.rank())
        val_dataset.sort_by_length()
        val_dataset._references = val_dataset._references[:batch_size*3]
        val_dataset._indices = np.arange(len(val_dataset))

        print(f'Group size: {hvd.size()} rank {hvd.rank()} got {len(val_dataset)} val batches')

    #opt_instance = tf.optimizers.Adam(initial_lr_global, beta_1=0.9, beta_2=0.999)
    opt_instance = tfa.optimizers.NovoGrad(initial_lr_global, beta_1=0.95, beta_2=0.5, weight_decay=0.001)

    opt = hvd.DistributedOptimizer(opt_instance)
    pipeline = get_pipeline(model, opt)
    
    callbacks = [
        hvd.callbacks.BroadcastGlobalVariablesCallback(0),
        hvd.callbacks.MetricAverageCallback(),
    ]
    #schedule=tf.keras.experimental.CosineDecayRestarts(
    #    initial_lr_global, 10, t_mul=1.0, m_mul=decay, alpha=0.0,
    #)
    schedule=tf.keras.experimental.CosineDecay(
        initial_lr_global, 134, alpha=decay
    )
    callbacks.append(LearningRateScheduler(schedule))
    if hvd.rank() == 0:
        prefix = datetime.now().strftime("%Y%m%d-%H%M%S")
        print('Will save to: {}'.format(os.path.join(model_dir, prefix)))
        monitor_metric_name = 'loss'  if not val_dataset_idx else 'val_loss'  # val_loss is wrong and broken
        callbacks.append(
            keras.callbacks.ModelCheckpoint(
                os.path.join(model_dir, prefix + '_best.h5'),
                monitor=monitor_metric_name, save_weights_only=True,
                save_best_only=True))
        if tensorboard:
            logdir = os.path.join(model_dir, 'tb', prefix)
            tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
            callbacks.append(tensorboard_callback)

    time_start = time.time()

    hist = pipeline.fit(dataset, epochs=epochs, dev_dataset=val_dataset,
                        #steps_per_epoch=270,
                        callbacks=callbacks,
                        verbose=1 if hvd.rank() == 0 else 0)
    elapsed = time.time() - time_start
    
    if hvd.rank() == 0:
        print(f'Elapsed time: {elapsed}')
        #np.save(os.path.join(model_dir, prefix + '_hist.p'), np.array(hist))

In [17]:
train_model(
    filename='./models/quartznet_5x5_mp_1ex.h5', 
    dataset_idx='./data/train-clean-360-index.csv',
    val_dataset_idx='./data/train-clean-360-index.csv',
    is_mixed_precision=True,
    batch_size=36,
    initial_lr=0.01,
    decay=0.01,
    epochs=1000,
    tensorboard=False,
    restart_filename='./models/quartznet_5x5_mp_1ex_train/20200619-132929_best.h5',
)

Group size: 1 rank 0 got 1 batches
Group size: 1 rank 0 got 3 val batches
Will save to: ./models/quartznet_5x5_mp_1ex_train/20200619-150412
Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 20.2030 - val_loss: 72.6288 - lr: 0.0100
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 20.2034 - val_loss: 72.6122 - lr: 0.0098
Epoch 3/1000
1/1 [==============================] - 1s 1s/step - loss: 20.2033 - val_loss: 72.6145 - lr: 0.0090
Epoch 4/1000
1/1 [==============================] - 1s 1s/step - loss: 20.2045 - val_loss: 72.6152 - lr: 0.0079
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 20.2032 - val_loss: 72.6063 - lr: 0.0065
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 20.2031 - val_loss: 72.6309 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 1s 1s/step - loss: 20.1869 - val_loss: 72.6509 - lr: 0.0035
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 20.17

KeyboardInterrupt: 

In [18]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description='Train ctc asr model')
    parser.add_argument('--filename', type=str,
                        help='filename of the model')
    parser.add_argument('--dataset', type=str,
                        help='path to the dataset index',)
    parser.add_argument('--val_dataset', type=str,
                        help='path to the validation dataset index (optional)',
                        default=None)
    parser.add_argument('--batch_size', type=int,
                       help='batch size for training and validation',
                       default=10)
    parser.add_argument('--lr', type=float,
                       help='initial learning rate',
                       default=0.005)
    parser.add_argument('--decay', type=float,
                       help='learning rate decay',
                       default=1.0)
    parser.add_argument('--epochs', type=int,
                       help='number of epochs to use for training',
                       default=25)
    parser.add_argument('--tensorboard', type=bool,
                       help='if tensorboard log will be written',
                       default=False)
    parser.add_argument('--restart_filename', type=str,
                       help='filename of the checkpoint to restart from',
                       default=None)
    parser.add_argument('--mp', type=bool,
                       help='if mixed precision training is requested',
                       default=False)
    args = parser.parse_args()
    
    train_model(filename=args.filename, dataset_idx=args.dataset,
                val_dataset_idx=args.val_dataset, epochs=args.epochs,
                tensorboard=args.tensorboard, restart_filename=args.restart_filename,
                is_mixed_precision=args.mp, initial_lr=args.lr, decay=args.decay)

usage: ipykernel_launcher.py [-h] [--filename FILENAME] [--dataset DATASET]
                             [--val_dataset VAL_DATASET]
                             [--batch_size BATCH_SIZE] [--lr LR]
                             [--decay DECAY] [--epochs EPOCHS]
                             [--tensorboard TENSORBOARD]
                             [--restart_filename RESTART_FILENAME] [--mp MP]
ipykernel_launcher.py: error: unrecognized arguments: -f /gpfs/data/home/r.schutski/.local/share/jupyter/runtime/kernel-490f6d0a-2b7f-4049-8e92-d4cf3c133fff.json


SystemExit: 2

/trinity/home/r.schutski/asr_speedup/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


training times: 25 epochs - 10 batch / epoch
 tt mixed - gpu - 9427s - batch = 1
 tt mixed - cpu - 29900s - batch = 1
 tt full  - cpu - 14050s - batch = 1

In [21]:
dataset = asr.dataset.Audio.from_csv('./data/train-clean-360-index.csv', batch_size=40,
                                         use_filesizes=True, group_size=1, rank=0)
dataset.sort_by_length()


In [22]:
a, _ = dataset.get_batch(10)

In [23]:
len(a[0])

34320

In [30]:
dataset._references = dataset._references[:batch_size*10]
dataset._indices = np.arange(len(dataset))

NameError: name 'batch_size' is not defined